In [6]:

import torch
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import sys
sys.path.append("../")
import D3QN


device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

class DuelingDeepQNetwork(nn.Module):
    def __init__(self, alpha, state_dim, action_dim, fc1_dim, fc2_dim):
        super(DuelingDeepQNetwork, self).__init__()

        self.fc1 = nn.Linear(state_dim, fc1_dim)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.V = nn.Linear(fc2_dim, 1)
        self.A = nn.Linear(fc2_dim, action_dim)

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.to(device)

    def forward(self, state):
        x = T.relu(self.fc1(state))
        x = T.relu(self.fc2(x))

        V = self.V(x)
        A = self.A(x)
        Q = V + A - T.mean(A, dim=-1, keepdim=True)

        return Q


cache agent

In [24]:
import time


def gen():
        L1_size = random.choice([4,8, 16, 32])
        L2_size = random.choice([128,256,512])
        L3_size = random.choice([500,600,700,1000])
        servie_size=random.uniform(0.05, 10)

        free_storage_size_ratio_L1 = random.uniform(0, 1)
        free_storage_size_ratio_L2 = random.uniform(0, 1)
        free_storage_size_ratio_L3 = random.uniform(0, 1)

        obs = {
            "es_load":random.uniform(0, 1),
            "free_storage_size_ratio_L1":random.uniform(0, 1),
            "free_storage_size_ratio_L2":random.uniform(0, 1),
            "free_storage_size_ratio_L3":random.uniform(0, 1),
            "can_L1_fit":free_storage_size_ratio_L1*servie_size<=L1_size,
            "can_L2_fit":free_storage_size_ratio_L2*servie_size<=L2_size,
            "can_L3_fit":free_storage_size_ratio_L3*servie_size<=L3_size,
            "servie_size":random.uniform(50, 1000),
            "estimated_fetch_time":random.uniform(1, 60),
            "is_popular":0,
            "charm":1,
            "service_request_frequency":0,
            "nearby_servers_count":0,
            "cached_in_nearby_servers":0,
            "es_request_frequency":0
        }
        return obs
data = list(gen().values())
data = torch.tensor(data).float()
model = torch.load('./stages/final_2/cache_agent/5/D3QN_q_eval.pth')
model.eval()



# 在推理之前，进行一次预热
with torch.no_grad():
    model.forward(data)

# 测量推理性能
num_iterations = 100000  # 你可以根据需要调整迭代次数

start_time = time.time()
for _ in range(num_iterations):
    with torch.no_grad():
        model.forward(data)
end_time = time.time()

# 计算每秒推理次数（IPS）
ips = num_iterations / (end_time - start_time)

print(f"Inference Per Second: {ips:.2f}")


Inference Per Second: 5614.21


maintance agent

In [25]:
import time


def gen():
        loc = "L1"
        obs = {
            "free_space_ratio":random.uniform(0.8, 1),
            "service_size_ratio":random.uniform(0, 0.1),
            "cached_in_L1": loc == "L1",
            "cached_in_L2": loc == "L2",
            "cached_in_L3": loc == "L3",
            "service_charm":random.randint(0, 5),
            "service_request_frequency":0,
            "es_request_frequency":random.uniform(0, 1),
            "es_cache_miss_rate":random.uniform(0.8, 1),
            "least_freq_index":random.randint(0, 2),
            "is_ugent":0
        }
        return obs
data = list(gen().values())
data = torch.tensor(data).float()
model = torch.load('./stages/final_2/maintainance_agent/5/D3QN_q_eval.pth')
model.eval()



# 在推理之前，进行一次预热
with torch.no_grad():
    model.forward(data)

# 测量推理性能
num_iterations = 100000  # 你可以根据需要调整迭代次数

start_time = time.time()
for _ in range(num_iterations):
    with torch.no_grad():
        model.forward(data)
end_time = time.time()

# 计算每秒推理次数（IPS）
ips = num_iterations / (end_time - start_time)

print(f"Inference Per Second: {ips:.2f}")


Inference Per Second: 5568.38
